In [60]:
import random
import math
import heapq
from collections import deque, defaultdict

In [61]:
#import nbimporter
#from fct_usuelles import lire_graphe, afficher_graphe, visualiser_graphe_par_etape
#from fct_usuelles import generate_cyclic_graph, generate_chain_graph, generate_spider_graph

prends en paramètre un graphe dans un état (càd avec des sommets brulés) et retourne le graphe dans l'état suivante

In [62]:
def successeurs(graphe, etat_actuel):
    """
    Génère l'état suivant en propageant la brûlure aux voisins des sommets déjà brûlés.
    :param graphe: Le graphe sous forme de dictionnaire.
    :param etat_actuel: Dictionnaire contenant l'état actuel des sommets (brûlés ou non brûlés).
    :return: Nouveau dictionnaire représentant l'état des sommets après propagation.
    """

    brulage = False  # Indique si un sommet a été brûlé dans cet appel de la fonction

    # Copier l'état actuel pour générer le nouvel état
    nouvel_etat = etat_actuel.copy()

    # Récupérer tous les sommets brûlés
    sommets_brules = [sommet for sommet, etat in nouvel_etat.items() if etat == 1]

    # Propager la brûlure aux voisins des sommets brûlés
    for sommet in sommets_brules:
        for voisin in graphe.get(sommet, []):  # Obtenir les voisins dans la liste d'adjacence
            if nouvel_etat[voisin] == 0:  # Brûler uniquement les sommets non brûlés
                nouvel_etat[voisin] = 1
                brulage = True

    return nouvel_etat, brulage

test_but : vérifie si tout les sommets sont brulées

In [63]:
def test_but(etat_actuel):
    """
    Vérifie si tous les sommets du graphe sont brûlés.
    :param etat_actuel: Dictionnaire contenant l'état actuel des sommets (brûlés ou non brûlés).
    :return: True si tous les sommets sont brûlés, False sinon.
    """
    return all(etat == 1 for etat in etat_actuel.values())


In [64]:
def choisir_sommet_a_bruler(graphe, etat_actuel):
    """
    Sélectionne un sommet non brûlé ayant le maximum de voisins.
    
    :param graphe: Le graphe sous forme de dictionnaire (liste d'adjacence).
    :param etat_actuel: Dictionnaire contenant l'état actuel des sommets.
    :return: Le sommet non brûlé avec le maximum de voisins, ou None si aucun sommet disponible.
    """
    # Trouver les sommets non brûlés
    sommets_non_brules = [sommet for sommet, etat in etat_actuel.items() if etat == 0]
    
    if not sommets_non_brules:
        # Aucun sommet non brûlé
        return None

    # Trouver le sommet avec le maximum de voisins
    sommet_max_voisins = max(sommets_non_brules, key=lambda sommet: len(graphe.get(sommet, [])))

    return sommet_max_voisins


In [65]:
def choisir_sommet_aleatoire(etat_actuel):
    sommets_non_brules = [sommet for sommet, etat in etat_actuel.items() if etat == 0]
    return random.choice(sommets_non_brules) if sommets_non_brules else None


## 1ère méthode

In [66]:
#def recherche_profondeur(graphe, etat_initial, successeurs, test_but):
def recherche_profondeur(graphe, etat_initial):
    """
    Algorithme générique de recherche pour brûler un graphe en choisissant un sommet par étape.
    :param graphe: Le graphe sous forme de dictionnaire (liste d'adjacence).
    :param etat_initial: Dictionnaire représentant l'état initial des sommets (brûlés ou non brûlés).
    :param successeurs: Fonction qui génère l'état suivant (propagation).
    :param test_but: Fonction qui vérifie si tous les sommets sont brûlés.
    :return: Chemin (liste des états successifs), sommets brûlés activement à chaque étape, et coût total.
    """
    # Initialisation : créer la liste des états à traiter
    etats_a_traiter = deque([{"etat": etat_initial, "cout": 1, "brules_actifs": []}])

    while etats_a_traiter:
        # Extraire un état
        noeud = etats_a_traiter.pop()

        noeud["etat"], brulage = successeurs(graphe, noeud["etat"])

        if brulage:
            noeud["cout"] += 1  # Chaque étape coûte 1

        # Vérifier si tous les sommets sont brûlés
        if test_but(noeud["etat"]):
            return noeud["brules_actifs"], noeud["cout"]

        # Choisir un nouveau sommet à brûler activement
        nouveau_sommet = choisir_sommet_a_bruler(graphe, noeud["etat"])
        #nouveau_sommet = choisir_sommet_aleatoire(noeud["etat"])
        if nouveau_sommet is None:
            # Si aucun sommet à brûler n'est disponible, retourner l'état actuel
            continue

        # Copier l'état courant
        etat_suivant = noeud["etat"].copy()

        # Marquer le sommet comme "brûlé"
        etat_suivant[nouveau_sommet] = 1

        # Ajouter le nouvel état à la liste des états à traiter
        etats_a_traiter.append({
            "etat": etat_suivant,
            "cout": noeud["cout"],  # Chaque étape coûte 1
            "brules_actifs": noeud["brules_actifs"] + [nouveau_sommet]
        })

    # Si aucun état final n'est trouvé
    return [], float("inf")


etape 1 :

parcours en largeur : popleft de la liste des états à traiter

parcours en largeur itéré : livre artificial intelligence section 3.4.4 et 3.4.5


etape 2 : couverture des balles

appliquer le A* dans le parcours

bruler ce noeuds pendant un certain temps
=> couverture par des balles

l'ensemble des action : 
le cout : la taille de la grande balle
fixer burning number dés le début (6)


# 2éme méthode

In [67]:
#def recherche_largeur(graphe, etat_initial, successeurs, test_but):
def recherche_largeur(graphe, etat_initial):
    """
    Algorithme générique de recherche en largeur pour brûler un graphe.
    À chaque étape, chaque sommet est exploré pour générer tous les états possibles.
    :param graphe: Le graphe sous forme de dictionnaire (liste d'adjacence).
    :param etat_initial: Dictionnaire représentant l'état initial des sommets (brûlés ou non brûlés).
    :param successeurs: Fonction qui génère l'état suivant (propagation).
    :param test_but: Fonction qui vérifie si tous les sommets sont brûlés.
    :return: Chemin (liste des états successifs), sommets brûlés activement à chaque étape, et coût total.
    """
    # Initialisation : file (FIFO) pour gérer les états à traiter
    etats_a_traiter = deque([{"etat": etat_initial, "cout": 1, "brules_actifs": []}])

    while etats_a_traiter:
        # Extraire un état de la file (FIFO)
        noeud = etats_a_traiter.popleft()

        # Propager l'état courant (brûlage des voisins déjà brûlés)
        noeud["etat"], brulage = successeurs(graphe, noeud["etat"])

        if brulage:
            noeud["cout"] += 1  # Chaque étape coûte 1

        # Vérifier si tous les sommets sont brûlés
        if test_but(noeud["etat"]):
            return noeud["brules_actifs"], noeud["cout"]

        # Générer tous les nouveaux états possibles en brûlant chaque sommet non brûlé
        for sommet in graphe:
            if noeud["etat"].get(sommet) != 1:  # Si le sommet n'est pas encore brûlé
                # Copier l'état courant
                etat_suivant = noeud["etat"].copy()

                # Marquer le sommet comme "brûlé"
                etat_suivant[sommet] = 1

                # Ajouter le nouvel état à la file
                etats_a_traiter.append({
                    "etat": etat_suivant,
                    "cout": noeud["cout"],
                    "brules_actifs": noeud["brules_actifs"] + [sommet]
                })

    # Si aucun état final n'est trouvé
    return [], float("inf")


In [68]:
# 10 sommets (3.16)
#fichier = r'instances\Stranke94\Stranke94.mtx'

# 29 sommets (5.38)
#fichier = r'instances\bn-mouse_visual-cortex_1\bn-mouse_visual-cortex_1.mtx'

# 34 sommets (5,83)
fichier = r'instances\karate\karate.mtx'

# 7057 sommets (84.005)
#fichier = r'instances\fb-pages-government\fb-pages-government.mtx'

# 11631 sommets (107.84)
#fichier = r'instances\web-wiki-crocodile\web-wiki-crocodile.mtx'

# 196 591 sommets (443.38)
#fichier = r'instances\loc-gowalla_edges\loc-gowalla_edges.mtx'

# Charger un graphe avec la fonction lire_graphe
graphe = lire_graphe(fichier)

In [69]:
# Graphe cyclique (4)
#graphe = generate_cyclic_graph(16)

# Graphe en chaîne (4)
#graphe = generate_chain_graph(16)

# Graphe en étoile/spider (4.58)
#graphe = generate_spider_graph(0, leg_length=4, num_legs=5)

In [70]:
#afficher_graphe(graphe)

In [71]:
#etat_initial = {sommet: 0 for sommet in graphe}

In [72]:
#sommets_actifs_p, cout = recherche_profondeur(graphe, etat_initial)
#
#print("*********** Parcours en profondeur ***********")
#print("Sommets brûlés à chaque étape:", sommets_actifs_p)
#print("Coût total:", cout)

In [73]:
#sommets_actifs_l, cout = recherche_largeur(graphe, etat_initial)
#
#print("*********** Parcours en largeur ***********")
#print("Sommets brûlés à chaque étape:", sommets_actifs_l)
#print("Coût total:", cout)

dire un sommet va etre brulé au 3 tour par exemple
notre solution (du 2eme algo) sera le  centre (sommet) de la boule ayant le rayon le plus grand

In [74]:
# Visualiser le graphe par étape
#visualiser_graphe_par_etape(graphe, sommets_actifs_l)


# Couverture des balles

### Définition clarifiée des balles :
1. **Rayon de la balle** :
   - Le rayon $ r $ détermine combien de niveaux de voisins (profondeur dans le graphe) la balle peut atteindre depuis son centre $ U $.
   - Par exemple :
     - Rayon 1 : Brûle uniquement les voisins directs de $ U $.
     - Rayon 2 : Brûle les voisins de $ U $ et les voisins des voisins de $ U $.

2. **Centre de la balle** :
   - Chaque balle a un centre $ U $, qui est un sommet du graphe. À partir de ce centre, la brûlure se propage jusqu’à une profondeur $ r $.

3. **Objectif** :
   - Trouver un ensemble de balles ($ U_i, r_i $) qui couvrent **tout le graphe**.
   - Chaque sommet du graphe doit être brûlé par au moins une balle.

4. **Contraintes** :
   - Les **rayons des balles doivent être différents**.
   - Toutes les combinaisons possibles de balles (centres et rayons) doivent être explorées pour garantir que le graphe est entièrement couvert.

5. **Propagation de la brûlure** :
   - La propagation de la brûlure à une profondeur $ r $ peut être simulée par un parcours en largeur (BFS) depuis le sommet $ U $, en limitant la profondeur de l’exploration à $ r $.

## Methode 1

In [75]:
def calculer_voisinage_etendu(graphe, u, distance):
    """Retourne tous les sommets atteignables depuis u en <= distance étapes."""
    visites = set()
    a_visiter = {u}
    for _ in range(distance):
        nouveaux_voisins = set()
        for v in a_visiter:
            if v not in visites:
                visites.add(v)
                nouveaux_voisins.update(graphe.get(v, []))
        a_visiter = nouveaux_voisins - visites
    return list(visites)

In [76]:
def trouver_sequence_brulage(graphe):
    n = len(graphe)
    max_rayon = math.ceil(math.sqrt(n))
    V = list(graphe.keys())

    # Précalcul des voisinages étendus pour chaque (sommet, rayon)
    voisinages = {}
    for u in V:
        voisinages[u] = {}
        for r in range(1, max_rayon + 1):
            voisinages[u][r] = calculer_voisinage_etendu(graphe, u, r)

    # État initial : aucun sommet brûlé, séquence vide, rayons utilisés vides
    queue = deque()
    queue.append((set(), [], set()))

    while queue:
        brules, sequence, rayons_utilises = queue.popleft()

        # Si tous les sommets sont brûlés, retourner la séquence
        if len(brules) == n:
            return sequence

        # Générer les actions possibles : seuls les sommets NON BRÛLÉS peuvent être des centres
        for v in V:
            if v not in brules:  # <-- Critère clé : v n'est pas déjà brûlé
                for r in range(1, max_rayon + 1):
                    if r not in rayons_utilises:
                        # Mettre à jour les sommets brûlés : v + son voisinage à distance <= r
                        nouveaux_brules = brules | set(voisinages[v][r])
                        nouvelle_sequence = sequence + [(v, r)]
                        nouveaux_rayons = rayons_utilises | {r}
                        queue.append((nouveaux_brules, nouvelle_sequence, nouveaux_rayons))

    return None  # Aucune solution trouvée

---

## Methode 2

In [77]:
def calculer_voisinage_etendu_beta(graphe, u, distance):
    """Calcule les sommets brûlés en BFS jusqu'à la distance donnée."""
    visited = {u}
    queue = {u}
    for _ in range(distance):
        next_queue = set()
        for v in queue:
            for neighbor in graphe[v]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    next_queue.add(neighbor)
        queue = next_queue
        if not queue:
            break
    return list(visited)

In [7]:
def trouver_sequence_optimale_beta(graphe):
    n = len(graphe)
    V = list(graphe.keys())
    best_sequence = None
    best_max_r = float('inf')
    best_num_balls = float('inf')

    # Mémoire pour éviter les états redondants (burned, used_radii) → (max_r, num_balls)
    memo = defaultdict(lambda: (float('inf'), float('inf')))

    # File de priorité : (heuristique, max_r, num_balls, burned, sequence, used_radii)
    heap = []
    heapq.heappush(heap, (0, 0, 0, set(), [], set()))

    while heap:
        heur, current_max_r, current_num, brules, seq, used_r = heapq.heappop(heap)

        # Élagage : ignorer les états pires que la meilleure solution actuelle
        if current_max_r > best_max_r or (current_max_r == best_max_r and current_num >= best_num_balls):
            continue

        # Mise à jour de la meilleure solution si tous les sommets sont brûlés
        if len(brules) == n:
            if current_max_r < best_max_r or (current_max_r == best_max_r and current_num < best_num_balls):
                best_max_r, best_num_balls, best_sequence = current_max_r, current_num, seq
            continue

        # Génération des actions possibles avec élagage dynamique
        for v in V:
            if v not in brules:
                max_possible_r = min(math.ceil(math.sqrt(n)), best_max_r - 1)
                for r in range(1, max_possible_r + 1):
                    if r in used_r:
                        continue

                    # Calcul du voisinage étendu
                    voisins = calculer_voisinage_etendu_beta(graphe, v, r)
                    new_burned = brules | set(voisins)
                    new_max_r = max(current_max_r, r)
                    new_num = current_num + 1

                    # Élagage mémoire : vérifier si cet état est déjà meilleur
                    key = (frozenset(new_burned), frozenset(used_r | {r}))
                    if memo[key][0] < new_max_r or (memo[key][0] == new_max_r and memo[key][1] <= new_num):
                        continue
                    memo[key] = (new_max_r, new_num)

                    # Heuristique : priorité = new_max_r + (nombre de sommets restants / n)
                    remaining = n - len(new_burned)
                    heuristic = new_max_r + (remaining / n)
                    heapq.heappush(heap, (heuristic, new_max_r, new_num, new_burned, seq + [(v, r)], used_r | {r}))

    return best_sequence

In [8]:
def trouver_sequence_optimale(graphe):
    n = len(graphe)
    V = list(graphe.keys())  # Vérifier que tous les éléments de V sont du même type
    best_sequence = None
    best_max_r = float('inf')
    best_num_balls = float('inf')

    # Précalcul des voisinages étendus
    voisinages = {}
    for u in V:
        voisinages[u] = {}
        max_rayon = math.ceil(math.sqrt(n))
        for r in range(1, max_rayon + 1):
            voisinages[u][r] = calculer_voisinage_etendu(graphe, u, r)

    # File de priorité : (heuristique, max_r, num_balls, brûlés, séquence, rayons_utilisés)
    heap = []
    heapq.heappush(heap, (0, 0, 0, set(), [], set()))

    # Mémoire pour éviter les redondances
    #memo = defaultdict(lambda: (float('inf'), float('inf')))

    while heap:
        heur, current_max_r, current_num, brules, seq, used_r = heapq.heappop(heap)

        # Élagage si une meilleure solution existe déjà
        if current_max_r > best_max_r or (current_max_r == best_max_r and current_num >= best_num_balls):
            continue

        # Solution trouvée
        if len(brules) == n:
            if current_max_r < best_max_r or (current_max_r == best_max_r and current_num < best_num_balls):
                best_max_r, best_num_balls, best_sequence = current_max_r, current_num, seq
            continue

        # Générer les prochaines actions
        for v in V:
            if v not in brules:
                max_possible_r = min(math.ceil(math.sqrt(n)), best_max_r)
                for r in range(1, max_possible_r + 1):
                    if r in used_r:
                        continue

                    # Calcul des nouveaux brûlés
                    nouveaux_brules = brules | set(voisinages[v][r])
                    new_max_r = max(current_max_r, r)
                    new_num = current_num + 1

                    # Élagage via mémoire
                    key = (frozenset(nouveaux_brules), frozenset(used_r | {r}))
                    if (new_max_r, new_num) >= memo[key]:
                        continue
                    memo[key] = (new_max_r, new_num)

                    # Heuristique pour prioriser les bons états
                    remaining = n - len(nouveaux_brules)
                    heuristic = new_max_r + (remaining / n)  # Priorité = rayon_max + progression
                    heapq.heappush(heap, (heuristic, new_max_r, new_num, nouveaux_brules, seq + [(v, r)], used_r | {r}))

    return best_sequence

In [9]:
#sequence = trouver_sequence_brulage(graphe)
#if sequence:
#    print("Solution optimale trouvée :", sequence)
#    liste_triee = sorted(sequence, key=lambda x: x[1], reverse=True)
#    centres = [centre for centre, rayon in liste_triee]
#else:
#    print("Aucune solution valide trouvée.")

In [10]:
#sequence = trouver_sequence_optimale_beta(graphe)
#if sequence:
#    print("Solution optimale trouvée :", sequence)
#    liste_triee = sorted(sequence, key=lambda x: x[1], reverse=True)
#    centres = [centre for centre, rayon in liste_triee]
#else:
#    print("Aucune solution valide trouvée.")


In [11]:
#sequence = trouver_sequence_optimale(graphe)
#if sequence:
#    print("Solution optimale trouvée :", sequence)
#    liste_triee = sorted(sequence, key=lambda x: x[1], reverse=True)
#    centres = [centre for centre, rayon in liste_triee]
#else:
#    print("Aucune solution valide trouvée.")


In [12]:
#visualiser_graphe_par_etape(graphe, centres)
